# **Installing the Required Packages**

In [ ]:
!pip install arabert
!pip install nltk
!pip install arabic_reshaper
!pip install python-bidi
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets
!pip install transformers

# **Importing the required Libraries and Tools**

In [ ]:
# Importing the necessary libraries
import os
import subprocess
import shutil
import nltk
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from nltk import word_tokenize
from google.colab import files
from google.colab import drive
from arabert.preprocess import ArabertPreprocessor
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import arabic_reshaper
from bidi.algorithm import get_display
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainerCallback, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback




# **Constructing a Dataframe from the Target data**

The Target Data about Citizen Reviews is provided in **.txt** format, where each review is available in one seperate **.txt** file and every category of reviews relevant to a particular government sector is stored in corresponding folder. So, we have 10 Folders in total represnting the 10 government sectors.

The following is required to be able to use it in this project:

1. Extract and Parse the text content from the **.txt** files within each folder.
2. Create and save a distinct **.csv** file for each folder (government sector) for easier future management.
3. Create a Merged Dataframe from the Sector **.csv** files and Optionally save it as one integrated **.csv** on Drive.

***Mounting Google Drive and defining the Data Folder Path***

In [ ]:
drive.mount('/content/drive', force_remount=True)



In [ ]:
# Defining the path to the main folder and the output folder

# Folder Main Path with the TXT Files
drive_main_folder_path = '/content/drive/My Drive/######'

# Folder Path for CSV Files for each Sector
drive_output_folder_path = '/content/drive/My Drive/######'

# Folder Path for one CSV File for all the Reviews (Merging All Secor CSV Files)
merged_reviews_drive_output_folder_path = '/content/drive/My Drive/######'

In [ ]:
# Function to process each category and save as a CSV file
def process_and_save_category(category_path, category_name, output_folder_path):
    data = {'Sector': [], 'Content': []}

    for file_name in os.listdir(category_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(category_path, file_name)
            with open(file_path, 'r') as file:
                content = file.read()
                data['Sector'].append(category_name)
                data['Content'].append(content)


    df = pd.DataFrame(data)
    output_file_path = os.path.join(output_folder_path, f"{category_name}.csv")
    df.to_csv(output_file_path, index=False)
    return df

In [ ]:
# Processing each category folder and saving a CSV file for each Government Sector in the specified folder
all_dataframes = []

for category in os.listdir(drive_main_folder_path):
    category_path = os.path.join(drive_main_folder_path, category)
    if os.path.isdir(category_path):
        df = process_and_save_category(category_path, category, drive_output_folder_path)
        all_dataframes.append(df)


In [ ]:
# Saving the merged DataFrame to a CSV file and Storing it in Google Drive


merged_reviews_df = pd.concat(all_dataframes, ignore_index=True)

merged_output_file_path = os.path.join(merged_reviews_drive_output_folder_path, '#######.csv') # Replace by actual dataset path
merged_reviews_df.to_csv(merged_output_file_path, index=False)



# **Data Understanding**

# **Data Inspection**

***Inspecting the Merged Dataset***

In [ ]:
df.duplicated(subset=['Content']).sum()

In [ ]:
# To set the display width for pandas DataFrame
pd.set_option('display.max_colwidth', 80)  # You can increase the number to widen the content

# getting number of rows and columns for the dataset
print(f"Number of Rows and Columns in Target Reviews Dataset: {merged_reviews_df.shape}\n")

# shuffle the dataset
merged_reviews_df = merged_reviews_df.sample(frac=1).reset_index(drop=True)

# Displaying the merged DataFrame

merged_reviews_df

In [ ]:
# getting number of rows and columns for the dataset

print(f"Number of Rows and Columns in Target Reviews Dataset: {merged_reviews_df.shape}")


In [ ]:
# Manually define the Arabic-to-English translation dictionary
translations = {
    'الاتصالات': 'Communication',
    'البنوك': 'Banking',
    'البيئة': 'Environment',
    'التعليم': 'Education',
    'التموين': 'Supply',
    'الزراعة': 'Agriculture',
    'الصحة': 'Healthcare',
    'القضاء': 'Judiciary',
    'الكهرباء': 'Electricity',
    'المياه والصرف الصحي': 'Water and Sanitation'}

In [ ]:
# getting unique number of Sectors and their names

print("No. of Unique Sectors: " + str(merged_reviews_df['Sector'].nunique()))
print("------------------------------------------------------------")
print("Unique Sectors: \n")
print(merged_reviews_df.Sector.unique())
print("------------------------------------------------------------")
print("English Traslation: \n")
list(translations.values())

In [ ]:
# getting the info of dataframes

print(merged_reviews_df.info())

**Comment:**
It is obvious that there is no Null values in any of the dataframes and the datset is 32.0+ KB which is a reasonable size for finetuning the ARBERT Model

In [ ]:
# get rows with duplicates

merged_reviews_df[merged_reviews_df.duplicated(keep=False)]


In [ ]:
# Getting a detailed Summary about the dataframe including the Categorical ones

merged_reviews_df.describe(include='all')


In [ ]:
# Checking duplictates and null values
print('Null Values: ', merged_reviews_df.isnull().sum(axis=0).sum(),
      '\n',
      'Duplicate Values: ', merged_reviews_df.duplicated(subset='Content').sum())


In [ ]:
#getting duplicate content

merged_reviews_df[merged_reviews_df.duplicated(subset=['Content'], keep=False)].sort_values(by='Content')

**Comment:**
No Duplicates or Null Values found

***Barplot showing the Reviews Distribution over Sectors***

In [ ]:
# Dictionary for Arabic to English translations

# Count the number of rows per sector
sector_counts = merged_reviews_df['Sector'].value_counts().reset_index()
sector_counts.columns = ['Sector', 'Count']

# Reverse the letters of each sector name for readability
sector_counts['Reversed_Sector'] = sector_counts['Sector'].apply(lambda x: get_display(arabic_reshaper.reshape(x)))
sector_counts['Label'] = sector_counts.apply(lambda x: f"{x['Reversed_Sector']} - {translations[x['Sector']]}", axis=1)


# Create the bar plot with Seaborn
plt.figure(figsize=(12, 8))
# Use a colorful palette
barplot = sns.barplot(x='Label', y='Count', data=sector_counts, palette='tab10')

# Annotate the bars with larger font size
for b in barplot.patches:
    barplot.annotate(format(b.get_height(), '.0f'),
               (b.get_x() + b.get_width() / 2., b.get_height()),
                ha='center', va='baseline', fontsize=12, color='black')  # Adjust fontsize here


# Set plot labels and title
plt.xlabel('Sector (Arabic - English)', fontsize=18)
plt.ylabel('Number of Reviews', fontsize=18)
plt.title('Distribution of Reviews Among Sectors', fontsize=20, fontweight='bold')

# set bar value font size

plt.xticks(fontsize=14, rotation=45, ha='right')
plt.yticks(fontsize=14)
plt.tight_layout()

# Show the plot
plt.show()

# **Data Preparation and Preprocessing**

## **Following amost the same light preprocessing applied in Further pretraining as recommeneded by scholars, we just added some important ones based on this dataset characteristics**

The Data Preprocessing Steps we need to apply for the integrated dataframe are the following:

1. Adjust Sentence and Paragraph spaces and line breaks.
2. Use AraBERT preprocess function to handle most of the preprocessing steps
3. Replacing ministry names with the general string Token. This normalization enabled the model to focus on the relevant context to classify reviews rather than relying on the explicit mention of the ministry name.
4. Use Specific Tokens for Relevant Companies: Replace company names with sector-specific tokens if those companies are closely tied to specific sectors.
5. Normalize the punctuations (convert Engligh to Arabic ones)
6. Selective Punctuation Removal: retain the following set {%-؟!.:؛،[]}

***Adjusting Sentence spaces and Paragraph line breaks.***

In [ ]:
def adjust_spaces_line_breaks(text):

    # Remove leading and trailing whitespace
    text = text.strip()

    # Replace multiple line breaks (indicating paragraphs) with a placeholder
    text = re.sub(r'\n\s*\n+', '\n\n', text)

    # Replace remaining single line breaks with a space
    text = re.sub(r'\n', ' ', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)


    return text

In [ ]:
# before applying the adjust_spaces_line_breaks function
merged_reviews_df['Content'].tolist()

In [ ]:
# applying the adjust_spaces_line_breaks function to the DataFrame

merged_reviews_df['Processed_Content'] = merged_reviews_df['Content'].apply(adjust_spaces_line_breaks)


In [ ]:
merged_reviews_df['Processed_Content'] = merged_reviews_df['Content'].apply(adjust_spaces_line_breaks)

In [ ]:
# after applying the adjust_spaces_line_breaks function

merged_reviews_df['Processed_Content'].tolist()

***AraBERT Preprocess Function***

In [ ]:
# Initialize AraBERT Preprocessor
model_name = "aubmindlab/bert-base-arabertv02"
arabert_prep = ArabertPreprocessor(model_name=model_name)

def arabert_preprocess(text):
    return arabert_prep.preprocess(text)

In [ ]:
# after applying the arabert_preprocess function

merged_reviews_df['Processed_Content'] = merged_reviews_df['Processed_Content'].apply(arabert_preprocess)
merged_reviews_df['Processed_Content'].tolist()

***Use Specific Tokens for Relevant Companies: Replace company names with sector-specific tokens if those companies are closely tied to specific sectors.***

In [ ]:

def replace_company_tokens(text, token_dict):
    pattern = re.compile(r'\b(' + '|'.join(map(re.escape, token_dict.keys())) + r')\b')
    return pattern.sub(lambda match: token_dict[match.group(0)], text)


In [ ]:
#  Replace any whole word occurrence of dictionary keys in the text with their corresponding values.

company_tokens = {
'مستشفى دراو المركزي': '[مؤسسة_أ]',
'مستشفى القباري': '[مؤسسة_ب]',
'مستشفى سعاد كفافي': '[مؤسسة_ج]',
'مستشفى منشية البكري': '[مؤسسة_د]',
'مستشفى العدوة المركزي': '[مؤسسة_ه]',
'مستشفى العياط': '[مؤسسة_ز]',
'مستشفى المنيرة': '[مؤسسة_ح]',
'مستشفى الفكرية المركزي': '[مؤسسة_م]',
'مستشفى القباري': '[مؤسسة_س]',
'مستشفى النيل': '[مؤسسة_ش]',
'مستشفى المبرة': '[مؤسسة_ف]',
'مستشفى المطرية': '[مؤسسة_ت]',
'مستشفى الحسين الجامعي': '[مؤسسة_خ]',
'مستشفى ٦ أكتوبر بالدقي': '[مؤسسة_ث]',
'معهد السكر': '[مؤسسة_ك]',
'مستشفى أطفال مصر': '[مؤسسة_ؤ]',
'مستشفى مدينة نصر': '[مؤسسة_ظ]',
'مستشفى الرمد الجديدة بكفر الشيخ': '[مؤسسة_غ]',
'مستشفى ٦ أكتوبر': '[مؤسسة_ى]',
'مستشفى شرق المدينه بالإسكندرية': '[مؤسسة_و]',
'مستشفى العديسات': '[مؤسسة_ع]',

'جامعة القاهرة':  '[مؤسسة_ق]',

'لمنظومة الشكاوى الموحد':'[مؤسسة_ن]',

'النجاح وأحمد عرابي ) بمركز بدر' : '[مؤسسة_ل]',
'مدرسة أبوحزام الإبتدائية التابعة لإدارة نجع حمادى التعليمية'  : '[مؤسسة_ة]',
'بشركة أبوقير': '[مؤسسة_ر]',

'جمعية الغريزات الزراعية بالغريزات مركز المراغة': '[مؤسسة_ذ]',
'جمعية الهيطة الزراعية التابعة لمركز صان الحجر' : '[مؤسسة_لا]',
'جمعية أبو طه الزراعية مركز بلقاس' : '[مؤسسة_أي]',

'البنك الأهلي': '[مؤسسة_عغ]',
'بنك الإمارات دبي الوطني' : '[مؤسسة_يس]',
'البنك التجاري الدولي' : '[مؤسسة_را]',
'بنك ناصر الاجتماعي' : '[مؤسسة_ون]',
'بنك ناصر' : '[مؤسسة_رض]',
'بنك وفا التجاري' : '[مؤسسة_دج]',
'بنك الإسكندرية' : '[مؤسسة_بك]',
'بنك الأهلي قطر' : '[مؤسسة_سش]',
'بالوحدة المحلية بفيشا الكبري وتتبع بنك مصر فرع منوف' : '[مؤسسة_حز]',
'البنك التجاري الدولي': '[مؤسسة_ضت]',
'CIB' : '[مؤسسة_قر]',
'cib' : '[مؤسسة_زو]',
'البنك الإمارات الوطني': '[مؤسسة_هك]',
'HSBC': '[مؤسسة_فش]',
'Abu Dhabi Commercial Bank': '[مؤسسة_هخ]',
'البنك المركزي': '[مؤسسة_يك]',

'وي': '[مؤسسة_غى]',
'أورانج': '[مؤسسة_عة]',
'فودافون': '[مؤسسة_سط]',
'الشركة المصرية للاتصالات': '[مؤسسة_ظب]'

}

In [ ]:

# apply the replace company_tokens function on the data
merged_reviews_df['Processed_Content'] = merged_reviews_df['Processed_Content'].apply(lambda x: replace_company_tokens(x, company_tokens))
merged_reviews_df['Processed_Content'].tolist()

In [ ]:
def replace_sector_names(text, token_dict):
    pattern = re.compile(r'\b(' + '|'.join(map(re.escape, token_dict.keys())) + r')\b')
    return pattern.sub(lambda match: token_dict[match.group(0)], text)


In [ ]:
sector_names = {
'وزارة الصحة': '[الوزارة]',
'وزيرة الصحة': '[الوزيرة]',
'وزير الصحة': '[الوزير]',


'وزارة الزراعة واستصلاح الأراضي':'[الوزارة]',
'وزيرة الزراعة واستصلاح الأراضي':'[الوزيرة]',
'وزير الزراعة واستصلاح الأراضي':'[الوزير]',


'وزارة الزراعة': '[الوزارة]',
'وزيرة الزراعة': '[الوزيرة]',
'وزير الزراعة': '[الوزير]',

'وزارة الري': '[الوزارة]',
'وزيرة الري': '[الوزيرة]',
'وزير الري': '[الوزير]',

'وزارة الموراد المائية والري': '[الوزارة]',
'وزيرة الموراد المائية والري': '[الوزيرة]',
'وزير الموراد المائية والري': '[الوزير]',

'وزارة التربية والتعليم': '[الوزارة]',
'وزيرة التربية والتعليم': '[الوزيرة]',
'وزير التربية والتعليم': '[الوزير]',

'وزارة التعليم العالي': '[الوزارة]',
'وزيرة التعليم العالي': '[الوزيرة]',
'وزير التعليم العالي': '[الوزير]',

'وزارة المياه والصرف الصحي': '[الوزارة]',
'وزيرة المياه والصرف الصحي': '[الوزيرة]',
'وزير المياه والصرف الصحي': '[الوزير]',

'وزارة الكهرباء': '[الوزارة]',
'وزيرة الكهرباء': '[الوزيرة]',
'وزير الكهرباء': '[الوزير]',

'وزارة العدل': '[الوزارة]',
'وزيرة العدل': '[الوزيرة]',
'وزير العدل': '[الوزير]',

'وزارة التموين': '[الوزارة]',
'وزيرة التموين': '[الوزيرة]',
'وزير التموين': '[الوزير]',

'وزارة البيئة': '[الوزارة]',
'وزيرة البيئة': '[الوزيرة]',
'وزير البيئة': '[الوزير]',


'وزارة الاتصالات وتكنولوجيا المعلومات': '[الوزارة]',
'وزيرة الاتصالات وتكنولوجيا المعلومات': '[الوزيرة]',
'وزير الاتصالات وتكنولوجيا المعلومات': '[الوزير]',

'وزارة الاتصالات': '[الوزارة]',
'وزيرة الاتصالات': '[الوزيرة]',
'وزير الاتصالات': '[الوزير]',


'لوزارة الصحة': '[الوزارة] ل',
'لوزيرة الصحة': '[الوزيرة] ل',
'لوزير الصحة': '[الوزير] ل',


'لوزارة الزراعة واستصلاح الأراضي':'[الوزارة] ل',
'لوزيرة الزراعة واستصلاح الأراضي':'[الوزيرة] ل',
'لوزير الزراعة واستصلاح الأراضي':'[الوزير] ل',

'لوزارة الزراعة': '[الوزارة] ل',
'لوزيرة الزراعة': '[الوزيرة] ل',
'لوزير الزراعة': '[الوزير] ل',


'لوزارة الري': '[الوزارة] ل',
'لوزيرة الري': '[الوزيرة] ل',
'لوزير الري': '[الوزير] ل',

'لوزارة المواد المائية والري': '[الوزارة] ل',
'لوزيرة المواد المائية والري': '[الوزيرة] ل',
'لوزير المواد المائية والري': '[الوزير] ل',

'لوزارة التربية والتعليم': '[الوزارة] ل',
'لوزيرة التربية والتعليم': '[الوزيرة] ل',
'لوزير التربية والتعليم': '[الوزير] ل',

'لوزارة التعليم العالي': '[الوزارة] ل',
'لوزيرة التعليم العالي': '[الوزيرة] ل',
'لوزير التعليم العالي': '[الوزير] ل',

'لوزارة المياه والصرف الصحي': '[الوزارة] ل',
'لوزيرة المياه والصرف الصحي': '[الوزيرة] ل',
'لوزير المياه والصرف الصحي': '[الوزير] ل',

'لوزارة الكهرباء': '[الوزارة] ل',
'لوزيرة الكهرباء': '[الوزيرة] ل',
'لوزير الكهرباء': '[الوزير] ل',

'لوزارة العدل': '[الوزارة] ل',
'لوزيرة العدل': '[الوزيرة] ل',
'لوزير العدل': '[الوزير] ل',

'لوزارة التموين': '[الوزارة] ل',
'لوزيرة التموين': '[الوزيرة] ل',
'لوزير التموين': '[الوزير] ل',

'لوزارة البيئة': '[الوزارة] ل',
'لوزيرة البيئة': '[الوزيرة] ل',
'لوزير البيئة': '[الوزير] ل',


'لوزارة الاتصالات وتكنولوجيا المعلومات': '[الوزارة] ل',
'لوزيرة الاتصالات وتكنولوجيا المعلومات': '[الوزيرة] ل',
'لوزير الاتصالات وتكنولوجيا المعلومات': '[الوزير] ل',

'لوزارة الاتصالات': '[الوزارة] ل',
'لوزيرة الاتصالات': '[الوزيرة] ل',
'لوزير الاتصالات': '[الوزير] ل',



'بوزارة الصحة': '[الوزارة] ب',
'بوزيرة الصحة': '[الوزيرة] ب',
'بوزير الصحة': '[الوزير] ب',


'بوزارة الزراعة واستصلاح الأراضي':'[الوزارة] ب',
'بوزيرة الزراعة واستصلاح الأراضي':'[الوزيرة] ب',
'بوزير الزراعة واستصلاح الأراضي':'[الوزير] ب',

'بوزارة الزراعة': '[الوزارة] ب',
'بوزيرة الزراعة': '[الوزيرة] ب',
'بوزير الزراعة': '[الوزير] ب',


'بوزارة الري': '[الوزارة] ب',
'بوزيرة الري': '[الوزيرة] ب',
'بوزير الري': '[الوزير] ب',

'بوزارة المواد المائية والري': '[الوزارة] ب',
'بوزيرة المواد المائية والري': '[الوزيرة] ب',
'بوزير المواد المائية والري': '[الوزير] ب',

'بوزارة التربية والتعليم': '[الوزارة] ب',
'بوزيرة التربية والتعليم': '[الوزيرة] ب',
'بوزير التربية والتعليم': '[الوزير] ب',

'بوزارة التعليم العالي': '[الوزارة] ب',
'بوزيرة التعليم العالي': '[الوزيرة] ب',
'بوزير التعليم العالي': '[الوزير] ب',

'بوزارة المياه والصرف الصحي': '[الوزارة] ب',
'بوزيرة المياه والصرف الصحي': '[الوزيرة] ب',
'بوزير المياه والصرف الصحي': '[الوزير] ب',

'بوزارة الكهرباء': '[الوزارة] ب',
'بوزيرة الكهرباء': '[الوزيرة] ب',
'بوزير الكهرباء': '[الوزير] ب',

'بوزارة العدل': '[الوزارة] ب',
'بوزيرة العدل': '[الوزيرة] ب',
'بوزير العدل': '[الوزير] ب',

'بوزارة التموين': '[الوزارة] ب',
'بوزيرة التموين': '[الوزيرة] ب',
'بوزير التموين': '[الوزير] ب',

'بوزارة البيئة': '[الوزارة] ب',
'بوزيرة البيئة': '[الوزيرة] ب',
'بوزير البيئة': '[الوزير] ب',


'بوزارة الاتصالات وتكنولوجيا المعلومات': '[الوزارة] ب',
'بوزيرة الاتصالات وتكنولوجيا المعلومات': '[الوزيرة] ب',
'بوزير الاتصالات وتكنولوجيا المعلومات': '[الوزير] ب',

'بوزارة الاتصالات': '[الوزارة] ب',
'بوزيرة الاتصالات': '[الوزيرة] ب',
'بوزير الاتصالات': '[الوزير] ب'


}

In [ ]:


# applying the replace_location_names function
merged_reviews_df['Processed_Content'] = merged_reviews_df['Processed_Content'].apply(lambda x: replace_sector_names(x, sector_names))
merged_reviews_df['Processed_Content'].tolist()


***Handling Punctuations***

In [ ]:
# Normalizing Punctuations (changing English to Arabic ones)
def normalize_punctuations(text):


  translation_table = str.maketrans({'.': '.', ',': '،', '?': '؟', '!': '!', ';': '؛', ':': '؛'})
  return text.translate(translation_table)



In [ ]:
# applying the normalize_punctuations function on merged_reviews_df

merged_reviews_df['Processed_Content'] = merged_reviews_df['Processed_Content'].apply(normalize_punctuations)
merged_reviews_df['Processed_Content'].tolist()

***Applying selective punctuation removal***

In [ ]:
# Defining selective_punctuation_removal function
def selective_punctuation_removal(text):
    allowed_punctuations = {'%', '٪','-', '؟', '!', '.', ':', '؛', '،', '[', ']'}
    pattern = re.compile(r'[^%\-\؟\!\.\:\؛\،\[\]\w\s0-9]')
    return pattern.sub('', text)


In [ ]:
# Applying selective_punctuation_removal function on merged_reviews_df

merged_reviews_df['Processed_Content'] = merged_reviews_df['Processed_Content'].apply(selective_punctuation_removal)
merged_reviews_df['Processed_Content'].tolist()

***Some Checks for Noisy Characters***

In [ ]:
# Investigating the English Words in the dataset

def contains_english_words(text):

    return bool(re.search(r'\b[A-Za-z]+\b', text))

# Print Reviews Content containing English words
reviews_with_english_words = merged_reviews_df[merged_reviews_df['Processed_Content'].astype(str).apply(contains_english_words)]['Processed_Content'].tolist()
reviews_with_english_words

In [ ]:
# dropping the Content Column (the one that is not preprocessed)

merged_reviews_df.drop(columns=['Content'], inplace=True)



# **Saving the Train and Test Data Splits to Google Drive for further usage in the Training and Evaluating SVM as well as Finetuning and Evaluating Both ARBERT Models**

In [ ]:
drive.mount('/content/drive')

# Save DataFrame to CSV
merged_reviews_df.to_csv('/content/#####', index=False)


# Destination file path on Drive
drive_destination_file = "/content/drive/My Drive/#####"

# Copy the file from the source path to the destination path
shutil.copy('/content/#####', drive_destination_file)

# **Splitting the Target Data for the Normal Scenario**

In [ ]:
drive.mount('/content/drive', force_remount=True)

drive_destination_file = "/content/drive/My Drive/#####"

# Reading the CSV file into a DataFrame
preprocessed_df = pd.read_csv(drive_destination_file)

# Displaying the DataFrame
preprocessed_df

In [ ]:
# Using the below function to split the data into train+val and test sets (80% train+val, 20% test) for the first scenario: Standard Splitting

def data_split(df, test_size, stratify_coulmn): # stratify_coulmn here is the Sector column
    train_val_data, test_data = train_test_split(df, test_size=test_size, stratify=stratify_coulmn, random_state=1)
    return train_val_data, test_data



In [ ]:
# Priniting the function output to check the sizes of the train and test splits

# Example usage:
train_data, test_data = data_split(preprocessed_df, 0.2, preprocessed_df['Sector'])

print("Loaded Train Set with Size:", len(train_data))
print("Loaded Test Set with Size:", len(test_data))

In [ ]:
# Shuffling data before saving for better models training and ensuring unbiased model input

train_data = train_data.sample(frac=1, random_state=1).reset_index(drop=True)
test_data = test_data.sample(frac=1, random_state=1).reset_index(drop=True)

##**Saving the Datasets Splits for the First Scenario**

In [ ]:
# Saving Training and Testing splits for the First Scenario
train_data_path = '/content/drive/My Drive/#####'
test_data_path = '/content/drive/My Drive/#####'


train_data.to_csv(train_data_path, index=False)
test_data.to_csv(test_data_path, index=False)

print(f"First Scenario Training data saved to {train_data_path}")
print(f"First Scenario Test data saved to {test_data_path}")

# Load the saved datasets from Google Drive
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

print("Loaded First Scenario Training Set with Size:", len(train_data))
print("Loaded First Scenario Test Set with Size:", len(test_data))


# **Splitting the Target Data for the Controlled Scenario**

In [ ]:
# Reading the CSV files for the chosen 250 most complex reviews as well as the remaining reviews, representing the normal ones

drive_destination_file_normal = "/content/drive/My Drive/#####"
drive_destination_file_complex = "/content/drive/My Drive/#####"

# Reading the CSV file into a DataFrame
preprocessed_df_normal = pd.read_csv(drive_destination_file_normal)
preprocessed_df_complex = pd.read_csv(drive_destination_file_complex)



***The following performs the split and verifies the distribution of complex reviews across sectors. It also calculates how many records need to be transferred from normal to complex test set to preserve class distribution***

In [ ]:
# Checking the data splits sizes of the complex data

cx_train_data, cx_test_data  = data_split(train_data, 0.7, preprocessed_df_complex['Sector'])

print("Loaded Complex Train Set with Size:", len(cx_train_data))
print("Loaded Complex Test Set with Size:", len(cx_test_data))


In [ ]:
# Calculating the number of records that should be in the testing data of this scenario to ensure (80% train+val, 20% test) splitting for the whole data

total_records_to_transfer_to_testing = test_data - cx_test_data
print(total_records_to_transfer_to_testing)

In [ ]:
# Checking the sector distribution in the test split of the complex data and making sure it is not exceeding that in the test split of the first scenario

print("Sector Distribution in the Test Split of the Complex Data: \n", cx_test_data['Sector'].value_counts())



In [ ]:
transfer_counts_per_sector = (test_data['Sector'].value_counts() - cx_test_data['Sector'].value_counts()).to_dict()


print("Specifying the Exact Number of Records to Transfer for each Sector to the Test Data Split in the Second Scenario: \n", transfer_counts_per_sector)


In [ ]:
# Function to transfer a specified number of records by sector independently

def transfer_records_by_sector(df1, df2, transfer_counts):
    # List to hold records to be transferred
    records_to_transfer = []

    for sector, count in transfer_counts.items():
        # Filter records by sector
        sector_df = df2[df2['Sector'] == sector]

        # Randomly Select the records to transfer, ensuring we do not transfer more than available records
        records = sector_df.sample(n=min(count, len(sector_df)), random_state=1)

        # Append to list
        records_to_transfer.append(records)

    # Concatenate all selected records
    records_to_transfer_df = pd.concat(records_to_transfer)

    # Remove selected records from df2
    df2 = df2.drop(records_to_transfer_df.index)

    # Append records to df1
    df1 = pd.concat([df1, records_to_transfer_df])

    return df1, df2



Sectors in **cx_test_data** with count exceeding that in the **test_data** split of the first scenario are indicated and extra records have been moved back to the **cx_train_data**. To keep the test size the same, these records were replaced by moving records from the **cx_train_data** to **cx_test_data**, but from other sectors, and distributed proportionally.


***The following is the Pseudocode for preparing the train and test splits for the Second Scenario:***

```

# Initialize list of sectors with excess records in test
Initialize excess_sectors_list = []

# Step 1: Identify problem sectors
FOR EACH (sector, value) IN transfer_counts_per_sector:

    
    IF value < 0:
        
        ADD sector TO excess_sectors_list
        
        PRINT "Sector [sector] has a negative count: [value]"

        
        # Balance excess in test set
        # Move ABS(value) records of this sector from cx_test_data to cx_train_data
        Call transfer_records_by_sector to move excess sector records from **cx_test_data** → **cx_train_data** after preparing the correct **transfer_counts** dictionary with ABS(value) no. of records of this sector

# Step 2: Maintain test size by refilling from other sectors
Calculate total_to_refill = Sum of all ABS(negative values)

# Step 3: Get list of eligible sectors
eligible_sectors = All unique sectors in cx_train_data EXCLUDING excess_sectors_list

# Step 4: Distribute total_to_refill proportionally among eligible_sectors
Construct refill_transfer_counts dictionary with only eligible sectors

# Step 5: Move records from cx_train_data to cx_test_data
Call transfer_records_by_sector to move records from other sectors in **cx_train_data** → **cx_test_data** with the **refill_transfer_counts** dictionary

# Step 6: Update the **transfer_counts_per_sector** dictionary based on updated
**cx_test_data** (if needed)
After the transfers, update or recalculate the sector distribution difference
between **cx_test_data** and the reference test_set of the first scenraio to verify balance was achieved.

# Step 7: Move remaining records from normal dataset to cx_test_data
Call transfer_records_by_sector to transfer records from **normal_data** → **cx_test_data** using the **transfer_counts_per_sector** dictionary to move records from the normal data to cx_test_data to complete the 20% test split.

# Step 8: Add remaining records in normal data to cx_train_data to finalize the training set.
Append all the remaining **normal_data** to **cx_train_data**
(Use pd.concat or call **transfer_records_by_sector** if tracking is needed)

```

In [ ]:
# Checking duplicates to ensure everything was done correctly

print(cx_train_data.duplicated().sum())
print(cx_test_data.duplicated().sum())

In [ ]:
# Checking the lengths of the updated cx_train_data and cx_train_data splits as well as their sector distriubtion before saving

print("Size of the Train Set in the Second Scenario:", len(cx_train_data))
print("Size of the Test Set in the Second Scenario:", len(cx_test_data))

print("Sector Distribution in the Train Split of the Second Scenario: \n", cx_train_data['Sector'].value_counts())
print("Sector Distribution in the Test Split of the Second Scenario: \n", cx_test_data['Sector'].value_counts())


In [ ]:
# Shuffling data before saving for better models training and ensuring unbiased model input

cx_train_data = cx_train_data.sample(frac=1, random_state=1).reset_index(drop=True)
cx_test_data = cx_test_data.sample(frac=1, random_state=1).reset_index(drop=True)


##**Saving the Datasets Splits for the Second Scenario**

In [ ]:
# Saving Training and Testing splits for the Second Scenario
cx_train_data_path = '/content/drive/My Drive/#####'
cx_test_data_path = '/content/drive/My Drive/#####'


cx_train_data.to_csv(cx_train_data_path, index=False)
cx_test_data.to_csv(cx_test_data_path, index=False)

print(f"Second Scenario Training data saved to {cx_train_data_path}")
print(f"Second Scenario Test data saved to {cx_test_data_path}")

# Load the saved datasets from Google Drive
cx_train_data = pd.read_csv(cx_train_data_path)
cx_test_data = pd.read_csv(cx_test_data_path)

print("Loaded Training Set with Noise Size:", len(cx_train_data))
print("Loaded Test Set with Noise Size:", len(cx_test_data))
